<a href="https://colab.research.google.com/github/sajadamouei/chip-seq-dataset-decoding/blob/main/main_decode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Directory structures

!pip install seedir # Python package that provides a visual representation of directory structures in the form of tree-like structures

import seedir as sd
sd.seedir('/path/to/parent/folder', style='emoji') # replace with the actual path to your parent folder

In [ ]:
# Read and show .bedGraph files

content = []
with open("/path/to/a/bedGraph/file/coverage.bedGraph")as f:
    for line in f:
        content.append(line.strip().split())

content

In [ ]:
# Read and show .bed files

content = []
with open("/path/to/a/bed/file/labels.bed")as f:
    for line in f:
        content.append(line.strip().split())

content

In [ ]:
# find subfolders and extract .gz files in the chip-seq dataset

import os, gzip, shutil

z=0
o=0
def gz_extract(directory):
    extension = ".gz"
    os.chdir(directory)
    for item in os.listdir(directory): # loop through items in dir
      if item.endswith(extension): # check for ".gz" extension
          gz_name = os.path.abspath(item) # get full path of files
          file_name = (os.path.basename(gz_name)).rsplit('.',1)[0] #get file name for file within
          with gzip.open(gz_name,"rb") as f_in, open(file_name,"wb") as f_out:
              shutil.copyfileobj(f_in, f_out)
#          os.remove(gz_name) # delete zipped file




for path, subdirs, files in os.walk('/content/drive/MyDrive/PhD/1-DEDProject/3-Chip-seq/H3K9me3_TDH_BP/samples'):
    for name in files:
      if (z%2) == 0:
        print(os.path.join(path))
        gz_extract(os.path.join(path))
        o+=1
      z+=1

print('Number of files = ',z)
print('Number of files extracted = ', o)

In [ ]:
import os

lst_ad = []
num = 0
co = 0

for path, subdirs, files in os.walk('C:/Users/47966/Desktop/delete/chip/H3K9me3_TDH_BP/samples'):
    for name in files:
        if (co%2) == 0:
            lst_ad.append(os.path.join(path))
            num+=1
        co+=1

print('Number of folders = ', num)

In [ ]:
# Decode coverage.bedgraph files

import numpy as np
import os

   
    
f_data = []
    
for l in range(len(lst_ad)): # Read the coverage and label files
    coverage = []
    with open(os.path.join(lst_ad[l],"coverage.bedGraph"))as f:
        for line in f:
            coverage.append(line.strip().split())

    labels = []
    with open(os.path.join(lst_ad[l],"labels.bed"))as f:
        for line in f:
            labels.append(line.strip().split())
        
    n = int(coverage[len(coverage)-1][2]) # Last genomic position number
    m = int(coverage[0][1]) # First genomic position number

    data = np.zeros(shape=(2,n-m)) # One array for each coverage file

    count = 0

    for i in range(len(coverage)): # Decode the encoded coverage file in a 2D arry
        for j in range(int(coverage[i][2])-int(coverage[i][1])):
            data[0,count] = int(coverage[i][1]) + j + 1
            data[1,count] = int(coverage[i][3])
            count += 1
        
    for i in range(len(labels)): # Save the labeled parts of coverage file based label file from the 2D array
            
        t, s_point = np.where(data == int(labels[i][1]))
        t, e_point = np.where(data == int(labels[i][2]))
        s_point = int(s_point)-5000 # Pick the labelled sequence with addtional  5000 lenght before start point
        e_point = int(e_point)+5000 # Pick the labelled sequence with addtional  5000 lenght after end point
        
        f_data.append(coverage[0][0])
        f_data.append(list(data[1,s_point:e_point]))
        f_data.append(labels[i][3])

    print(l)

In [ ]:
import pickle # Save decoded and labeled dataset in a list .ob file
with open('C:/Users/47966/Desktop/delete/chip/f_data2.ob', 'wb') as fp:
    pickle.dump(f_data, fp)